<a href="https://colab.research.google.com/github/Precillieo/dsn20/blob/master/LGBM_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DSN** **2020**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, log_loss, roc_auc_score, accuracy_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report 

from lightgbm import LGBMClassifier
!pip install catboost
!pip install xgboost
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import warnings


     |████████████████████████████████| 66.1MB 57kB/s 


In [2]:
train= pd.read_csv('/content/drive/My Drive/Bootcamp2020/Train.csv')
test= pd.read_csv("/content/drive/My Drive/Bootcamp2020/Test.csv")

In [3]:
train.head(10)

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.0000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.0000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.240,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.2820,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.0000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.000,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.0000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.190,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no
5,Apcnt_1000020,3724.0,NaN,0.0000,0.0,0.0000,192944.0,9244585.0,6761209.0,67963357.0,32902009.0,NaN,976390.0,2114215.0,16576457,NaN,0.0,0.0,0.0,0.0,0.0,8.385,NaN,NaN,4.845,2590.5,22083.0,14722.0,3772.0,22083.0,NaN,NaN,3858.0,31.0000,0.0,NaN,10.0,60.0,0.0,0.0,NaN,NaN,0.030558,12.12,0.382960,NaN,0.0,lending,NaN,0.000000,NaN,no
6,Apcnt_1000024,3350.0,0.24205,0.3132,0.0,2.8188,NaN,580388.0,NaN,96472.0,580388.0,631347.0,NaN,0.0,21973443,91602498.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,822.0,NaN,NaN,426.0,13413.0,NaN,13413.0,36.0,14.3334,0.0,NaN,12.0,2.0,2.0,0.0,153.8745,NaN,0.660000,1.01,0.513504,NaN,0.0,charge,NaN,2.146512,0.004390,yes
7,Apcnt_1000028,3416.0,0.87020,0.0570,0.0,0.0000,17505.0,186331.0,186331.0,1484813.0,326760.0,50959.0,1284089.0,5633109.0,11564970,18211813.0,0.0,0.0,0.0,1.0,1.0,124.980,8267.655,7755.00,82.680,1498.5,1734.0,1156.0,548.0,15240.0,182.0,NaN,460.0,5.5000,0.0,1.0,10.0,6.0,12.0,0.0,NaN,149.0190,0.137500,3.03,0.635960,NaN,0.0,charge,155.295994,0.946074,0.128389,no
8,Apcnt_1000036,3376.0,0.29180,0.5586,0.0,0.0000,18283.0,1105149.0,18283.0,115533.0,1557167.0,752715.0,97639.0,427511.0,11564970,NaN,0.0,0.0,0.0,1.0,2.0,126.495,NaN,NaN,126.495,1597.5,3195.0,2130.0,182.0,3741.0,1034.0,NaN,70.0,34.8334,1.0,NaN,14.0,0.0,8.0,0.0,NaN,102.2625,0.471427,4.04,0.410880,0.0,0.0,charge,NaN,0.641601,0.009990,no
9,Apcnt_1000040,3310.0,0.24395,0.1562,0.0,0.0000,NaN,0.0,NaN,NaN,0.0,208115.0,NaN,NaN,11564970,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,502.5,1278.0,852.0,182.0,9126.0,NaN,273.0,22.0,4.1666,0.0,1.0,4.0,0.0,4.0,0.0,150.0000,NaN,0.825000,1.01,0.800000,NaN,NaN,charge

In [4]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,0.0,1649749.0,5446.0,0.0,48187375,114686147.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1086.0,6204.0,2676.0,60.0,14691.0,9794.0,NaN,238.0,1.8334,0.0,NaN,22.0,2.0,14.0,0.0,NaN,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,622789.0,1025793.0,1396510.0,34694910,87479487.0,1.0,1.0,1.0,2.0,2.0,142.560,NaN,NaN,570.585,3082.5,10221.0,6814.0,60.0,11955.0,3528.0,NaN,1216.0,19.5000,3.0,NaN,30.0,18.0,14.0,2.0,NaN,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,0.0,118256.0,NaN,NaN,21973443,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.0,4836.0,NaN,NaN,NaN,7.8334,0.0,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,493641.0,1391064.0,1176725.0,1220293.0,16576457,NaN,2.0,2.0,2.0,3.0,3.0,109.005,13050.00,NaN,109.005,1414.5,4197.0,2798.0,60.0,14964.0,NaN,NaN,248.0,9.0000,1.0,1.0,36.0,0.0,26.0,0.0,NaN,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,271133.0,613064.0,4779643.0,48187375,72049802.0,0.0,0.0,0.0,3.0,3.0,139.065,469.29,469.29,15.180,2724.0,9855.0,6570.0,182.0,11679.0,5900.0,NaN,2388.0,16.6666,1.0,0.0,24.0,28.0,20.0,0.0,NaN,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 51 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Applicant_ID  24000 non-null  object 
 1   form_field1   22890 non-null  float64
 2   form_field2   22291 non-null  float64
 3   form_field3   23854 non-null  float64
 4   form_field4   23854 non-null  float64
 5   form_field5   23854 non-null  float64
 6   form_field6   18396 non-null  float64
 7   form_field7   21769 non-null  float64
 8   form_field8   18396 non-null  float64
 9   form_field9   20600 non-null  float64
 10  form_field10  23853 non-null  float64
 11  form_field11  10602 non-null  float64
 12  form_field12  19817 non-null  float64
 13  form_field13  21537 non-null  float64
 14  form_field14  24000 non-null  int64  
 15  form_field15  14408 non-null  float64
 16  form_field16  18526 non-null  float64
 17  form_field17  19305 non-null  float64
 18  form_field18  19631 non-nu

In [7]:
train.isnull().sum()

Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

In [8]:
test.isnull().sum()

Applicant_ID        0
form_field1      1110
form_field2      1709
form_field3       146
form_field4       146
form_field5       146
form_field6      5604
form_field7      2231
form_field8      5604
form_field9      3400
form_field10      147
form_field11    13398
form_field12     4183
form_field13     2463
form_field14        0
form_field15     9592
form_field16     5474
form_field17     4695
form_field18     4369
form_field19        0
form_field20      147
form_field21     6707
form_field22     8724
form_field23    12125
form_field24     5605
form_field25     2256
form_field26     3172
form_field27     3910
form_field28      147
form_field29      147
form_field30    10908
form_field31    16810
form_field32     2256
form_field33      495
form_field34      147
form_field35     9866
form_field36      903
form_field37     2256
form_field38      147
form_field39     1829
form_field40    18828
form_field41    16349
form_field42      578
form_field43      250
form_field44     2362
form_field

In [9]:
train.columns

Index(['Applicant_ID', 'form_field1', 'form_field2', 'form_field3',
       'form_field4', 'form_field5', 'form_field6', 'form_field7',
       'form_field8', 'form_field9', 'form_field10', 'form_field11',
       'form_field12', 'form_field13', 'form_field14', 'form_field15',
       'form_field16', 'form_field17', 'form_field18', 'form_field19',
       'form_field20', 'form_field21', 'form_field22', 'form_field23',
       'form_field24', 'form_field25', 'form_field26', 'form_field27',
       'form_field28', 'form_field29', 'form_field30', 'form_field31',
       'form_field32', 'form_field33', 'form_field34', 'form_field35',
       'form_field36', 'form_field37', 'form_field38', 'form_field39',
       'form_field40', 'form_field41', 'form_field42', 'form_field43',
       'form_field44', 'form_field45', 'form_field46', 'form_field47',
       'form_field48', 'form_field49', 'form_field50', 'default_status'],
      dtype='object')

In [10]:
test.columns

Index(['Applicant_ID', 'form_field1', 'form_field2', 'form_field3',
       'form_field4', 'form_field5', 'form_field6', 'form_field7',
       'form_field8', 'form_field9', 'form_field10', 'form_field11',
       'form_field12', 'form_field13', 'form_field14', 'form_field15',
       'form_field16', 'form_field17', 'form_field18', 'form_field19',
       'form_field20', 'form_field21', 'form_field22', 'form_field23',
       'form_field24', 'form_field25', 'form_field26', 'form_field27',
       'form_field28', 'form_field29', 'form_field30', 'form_field31',
       'form_field32', 'form_field33', 'form_field34', 'form_field35',
       'form_field36', 'form_field37', 'form_field38', 'form_field39',
       'form_field40', 'form_field41', 'form_field42', 'form_field43',
       'form_field44', 'form_field45', 'form_field46', 'form_field47',
       'form_field48', 'form_field49', 'form_field50'],
      dtype='object')

In [11]:
train.describe()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field48,form_field49,form_field50
count,53471.000000,52156.000000,55645.000000,55645.000000,55645.000000,4.264000e+04,5.083700e+04,4.264000e+04,4.799200e+04,5.564500e+04,2.457900e+04,4.610500e+04,5.011100e+04,5.600000e+04,3.352500e+04,42964.000000,44849.000000,45598.000000,55996.000000,55645.000000,40146.000000,35600.000000,27877.000000,42703.000000,50550.000000,48562.000000,46701.000000,55645.000000,55645.000000,30491.000000,16592.000000,50550.000000,54744.000000,55645.000000,32852.000000,54005.000000,50550.000000,55645.000000,51789.000000,12271.000000,17771.000000,54677.000000,55432.000000,50617.000000,24683.000000,40096.000000,3.511100e+04,55645.000000,4.494400e+04
mean,3491.795665,0.550737,1.052225,0.851979,1.956317,6.244479e+05,6.865210e+06,2.626690e+06,1.316002e+07,1.185585e+07,2.054715e+06,2.771113e+06,6.009944e+06,6.914875e+07,1.145041e+08,0.230425,0.265692,0.271832,0.948800,1.031593,86.500380,6628.316381,6452.875072,104.558516,2624.555935,10106.554899,6190.516948,17274.315141,16518.540695,5213.246663,10857.565755,938.345321,13.926583,0.230245,0.199866,12.859069,8.480435,8.601420,0.090849,147.797977,108.117363,0.368215,6.634511,0.563377,0.072520,0.095371,3.052449e+05,1.049061,6.005862e+05
std,188.462426,0.820979,2.147768,3.157692,10.512396,1.433422e+06,1.912729e+07,3.927355e+06,1.977963e+07,2.669459e+07,4.912980e+06,4.544838e+06,2.335893e+07,2.713995e+09,1.153842e+08,0.621833,0.720198,0.725434,1.736454,1.785122,53.445528,5661.059297,5739.968774,436.670719,1964.226548,8796.472481,5390.915344,26930.742020,9666.183962,3124.491488,6049.678489,1044.597044,15.208396,0.683063,0.574002,10.621984,9.328810,7.969215,0.423802,45.085889,36.765769,0.412858,6.378946,0.196973,0.291146,0.335210,1.647757e+06,1.806160,5.842405e+06
min,2990.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.395430e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.255000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166600,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.021000,0.000000,0.000000,0.000000,0.048528,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,3358.000000,0.070788,0.000000,0.000000,0.000000,1.400400e+04,6.869740e+05,1.929440e+05,1.368502e+06,4.500730e+05,2.172565e+05,2.758010e+05,1.921660e+05,1.657646e+07,5.007636e+07,0.000000,0.000000,0.000000,0.000000,0.000000,40.305000,1068.667500,797.145000,14.865000,1231.500000,3375.000000,2068.000000,60.000000,9765.000000,2312.000000,6297.000000,194.000000,2.333400,0.000000,0.000000,6.000000,2.000000,4.000000,0.000000,136.191750,87.055500,0.000000,2.020000,0.411672,0.000000,0.000000,7.710079e+01,0.000000,4.560141e-02
50%,3484.000000,0.267575,0.062000,0.000000,0.000000,1.155330e+05,2.704328e+06,9.639420e+05,5.506295e+06,3.707559e+06,6.986440e+05,1.163110e+06,1.605792e+06,2.814143e+07,8.442428e+07,0.000000,0.000000,0.000000,0.000000,0.000000,89.167500,5976.225000,5548.935000,60.660000,2203.500000,8214.000000,5110.000000,486.000000,15057.000000,5658.000000,10584.000000,598.000000,7.833400,0.000000,0.000000,10.000000,6.000000,6.000000,0.000000,150.000000,118.441500,0.220000,5.050000,0.588648,0.000000,0.000000,3.181243e+02,0.000000,1.703996e-01
75%,3620.000000,0.719512,1.282000,0.000000,0.000000,5.259280e+05,6.993831e+06,3.751516e+06,1.694552

## **DEALING** **WITH** **MISSING** **VALUES**

In [12]:
columns= ('form_field1', 'form_field2', 'form_field3', 'form_field4', 'form_field5', 'form_field6', 'form_field7', 'form_field8', 'form_field9', 'form_field10', 'form_field11', 'form_field12', 'form_field13', 'form_field17', 'form_field15', 'form_field16', 'form_field18', 'form_field19', 'form_field20', 'form_field21', 'form_field22', 'form_field23', 'form_field24', 'form_field25', 'form_field26', 'form_field27', 'form_field28', 'form_field29', 'form_field30', 'form_field31', 'form_field32', 'form_field33', 'form_field34', 'form_field35', 'form_field36', 'form_field37', 'form_field38', 'form_field39', 'form_field40', 'form_field41', 'form_field42', 'form_field43', 'form_field44', 'form_field45', 'form_field46', 'form_field48','form_field49','form_field50')
for cod in columns:
  test[cod]= test[cod].fillna(0)

test.isnull().sum()

Applicant_ID    0
form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [13]:
cols= ('form_field1', 'form_field2', 'form_field3', 'form_field4', 'form_field5', 'form_field6', 'form_field7', 'form_field8', 'form_field9', 'form_field10', 'form_field11', 'form_field12', 'form_field13', 'form_field17', 'form_field15', 'form_field16', 'form_field18', 'form_field19', 'form_field20', 'form_field21', 'form_field22', 'form_field23', 'form_field24', 'form_field25', 'form_field26', 'form_field27', 'form_field28', 'form_field29', 'form_field30', 'form_field31', 'form_field32', 'form_field33', 'form_field34', 'form_field35', 'form_field36', 'form_field37', 'form_field38', 'form_field39', 'form_field40', 'form_field41', 'form_field42', 'form_field43', 'form_field44', 'form_field45', 'form_field46', 'form_field48','form_field49','form_field50')
for col in cols:
  train[col]= train[col].fillna(0)

train.isnull().sum()

Applicant_ID      0
form_field1       0
form_field2       0
form_field3       0
form_field4       0
form_field5       0
form_field6       0
form_field7       0
form_field8       0
form_field9       0
form_field10      0
form_field11      0
form_field12      0
form_field13      0
form_field14      0
form_field15      0
form_field16      0
form_field17      0
form_field18      0
form_field19      0
form_field20      0
form_field21      0
form_field22      0
form_field23      0
form_field24      0
form_field25      0
form_field26      0
form_field27      0
form_field28      0
form_field29      0
form_field30      0
form_field31      0
form_field32      0
form_field33      0
form_field34      0
form_field35      0
form_field36      0
form_field37      0
form_field38      0
form_field39      0
form_field40      0
form_field41      0
form_field42      0
form_field43      0
form_field44      0
form_field45      0
form_field46      0
form_field47      0
form_field48      0
form_field49      0


In [14]:
train.dtypes

Applicant_ID       object
form_field1       float64
form_field2       float64
form_field3       float64
form_field4       float64
form_field5       float64
form_field6       float64
form_field7       float64
form_field8       float64
form_field9       float64
form_field10      float64
form_field11      float64
form_field12      float64
form_field13      float64
form_field14        int64
form_field15      float64
form_field16      float64
form_field17      float64
form_field18      float64
form_field19      float64
form_field20      float64
form_field21      float64
form_field22      float64
form_field23      float64
form_field24      float64
form_field25      float64
form_field26      float64
form_field27      float64
form_field28      float64
form_field29      float64
form_field30      float64
form_field31      float64
form_field32      float64
form_field33      float64
form_field34      float64
form_field35      float64
form_field36      float64
form_field37      float64
form_field38

In [15]:
test.dtypes

Applicant_ID     object
form_field1     float64
form_field2     float64
form_field3     float64
form_field4     float64
form_field5     float64
form_field6     float64
form_field7     float64
form_field8     float64
form_field9     float64
form_field10    float64
form_field11    float64
form_field12    float64
form_field13    float64
form_field14      int64
form_field15    float64
form_field16    float64
form_field17    float64
form_field18    float64
form_field19    float64
form_field20    float64
form_field21    float64
form_field22    float64
form_field23    float64
form_field24    float64
form_field25    float64
form_field26    float64
form_field27    float64
form_field28    float64
form_field29    float64
form_field30    float64
form_field31    float64
form_field32    float64
form_field33    float64
form_field34    float64
form_field35    float64
form_field36    float64
form_field37    float64
form_field38    float64
form_field39    float64
form_field40    float64
form_field41    

In [16]:
cat= ('form_field47', 'Applicant_ID', 'default_status')
for x in cat:
  le= LabelEncoder()
  train[x]= le.fit_transform(train[x].values)
  

In [17]:
cats= ('form_field47', 'Applicant_ID')
for y in cats:
  le= LabelEncoder()
  test[y]= le.fit_transform(test[y].values)
  

## **FEATURE** **ENGINEERING**

In [18]:
train.dtypes

Applicant_ID        int64
form_field1       float64
form_field2       float64
form_field3       float64
form_field4       float64
form_field5       float64
form_field6       float64
form_field7       float64
form_field8       float64
form_field9       float64
form_field10      float64
form_field11      float64
form_field12      float64
form_field13      float64
form_field14        int64
form_field15      float64
form_field16      float64
form_field17      float64
form_field18      float64
form_field19      float64
form_field20      float64
form_field21      float64
form_field22      float64
form_field23      float64
form_field24      float64
form_field25      float64
form_field26      float64
form_field27      float64
form_field28      float64
form_field29      float64
form_field30      float64
form_field31      float64
form_field32      float64
form_field33      float64
form_field34      float64
form_field35      float64
form_field36      float64
form_field37      float64
form_field38

In [19]:
train['form_field3233']= (train.form_field32 + train.form_field33) / 2
train.form_field3233

0         530.0000
1        1285.7500
2          35.3333
3         264.9167
4         136.5000
           ...    
55995     442.8333
55996     128.3333
55997     127.1667
55998      80.7500
55999     705.4167
Name: form_field3233, Length: 56000, dtype: float64

In [20]:
test['form_field3233']= (test.form_field32 + test.form_field33) / 2
test.form_field3233

0         119.9167
1         617.7500
2           3.9167
3         128.5000
4        1202.3333
           ...    
23995      12.1667
23996     300.0833
23997     103.6667
23998      32.3333
23999     829.4167
Name: form_field3233, Length: 24000, dtype: float64

In [21]:
train= train.drop(['form_field32', 'form_field33'], axis=1)
test= test.drop(['form_field32', 'form_field33'], axis= 1)

## **MODELLING**

In [22]:
X= train.drop(['default_status'], axis=1)
y= train['default_status']


In [23]:
train_x, val_x, train_y, val_y= train_test_split(X, y, test_size= 0.3,stratify= y, random_state= 42)

In [24]:
le= LogisticRegression()

rf= RandomForestClassifier(n_estimators=700, random_state=0)
ab= AdaBoostClassifier(n_estimators=900, learning_rate=0.1, random_state= 0)
gb= GradientBoostingClassifier(n_estimators=600,learning_rate=0.1,random_state=0)

In [25]:
le.fit(train_x, train_y)
y_pred = le.predict(val_x)
pred= le.predict_proba(val_x)
print('le:', roc_auc_score(val_y, pred[:,1]))
bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
print(bin_clf_rep)
cv_s = cross_val_score(le,X, y ,cv=5).mean()
print(cv_s)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


le: 0.7428153183689414
              precision    recall  f1-score   support

           0       0.76      0.99      0.86     12686
           1       0.47      0.01      0.03      4114

    accuracy                           0.75     16800
   macro avg       0.61      0.50      0.44     16800
weighted avg       0.69      0.75      0.66     16800



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.7568571428571429


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [26]:
rf.fit(train_x, train_y)
pred= rf.predict_proba(val_x)
y_pred = rf.predict(val_x)
print('rf:', roc_auc_score(val_y, pred[:,1]))
cv_sc = cross_val_score(rf,X, y ,cv=5).mean()
bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
print(bin_clf_rep)
print(cv_sc)

rf: 0.8337228093609292
              precision    recall  f1-score   support

           0       0.83      0.94      0.88     12686
           1       0.68      0.43      0.53      4114

    accuracy                           0.81     16800
   macro avg       0.76      0.68      0.70     16800
weighted avg       0.80      0.81      0.79     16800

0.7971964285714286


In [27]:
ab.fit(train_x, train_y)
pred= ab.predict_proba(val_x)
y_pred = ab.predict(val_x)
print('ab:', roc_auc_score(val_y, pred[:,1]))
bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
cv_sco= cross_val_score(ab,X, y ,cv=5).mean()
print(cv_sco)
print(bin_clf_rep)

ab: 0.8361620793817935
0.7791428571428571
              precision    recall  f1-score   support

           0       0.84      0.93      0.88     12686
           1       0.67      0.44      0.53      4114

    accuracy                           0.81     16800
   macro avg       0.75      0.68      0.71     16800
weighted avg       0.80      0.81      0.79     16800



In [30]:
gb.fit(train_x, train_y)
pred= gb.predict_proba(val_x)
y_pred = gb.predict(val_x)
print('gb:', roc_auc_score(val_y, pred[:,1]))
bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
cv_scor = cross_val_score(gb,X, y ,cv=5).mean()
print(cv_scor)
print(bin_clf_rep)

gb: 0.8382422302852084
0.6968928571428572
              precision    recall  f1-score   support

           0       0.84      0.92      0.88     12686
           1       0.66      0.46      0.54      4114

    accuracy                           0.81     16800
   macro avg       0.75      0.69      0.71     16800
weighted avg       0.80      0.81      0.80     16800



In [32]:
xb= XGBClassifier()
xb.fit(train_x, train_y)
pred= xb.predict_proba(val_x)
y_pred = xb.predict(val_x)
print('xb:', roc_auc_score(val_y, pred[:,1]))
bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
cv_scor = cross_val_score(xb,X, y ,cv=5).mean()
print(cv_scor)
print(bin_clf_rep)

xb: 0.8355494260953644
0.7776071428571429
              precision    recall  f1-score   support

           0       0.84      0.93      0.88     12686
           1       0.67      0.44      0.53      4114

    accuracy                           0.81     16800
   macro avg       0.75      0.68      0.70     16800
weighted avg       0.79      0.81      0.79     16800



In [35]:
cb=CatBoostClassifier()
cb.fit(train_x, train_y)
pred= cb.predict_proba(val_x)
y_pred = cb.predict(val_x)
print('cb:', roc_auc_score(val_y, pred[:,1]))
bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
cv_scor = cross_val_score(cb,X, y ,cv=5).mean()
print(cv_scor)
print(bin_clf_rep)

Streaming output truncated to the last 5000 lines.
15:	learn: 0.4665953	total: 650ms	remaining: 40s
16:	learn: 0.4625844	total: 689ms	remaining: 39.8s
17:	learn: 0.4584501	total: 727ms	remaining: 39.7s
18:	learn: 0.4547511	total: 770ms	remaining: 39.8s
19:	learn: 0.4515566	total: 808ms	remaining: 39.6s
20:	learn: 0.4486310	total: 844ms	remaining: 39.4s
21:	learn: 0.4458942	total: 887ms	remaining: 39.4s
22:	learn: 0.4432536	total: 925ms	remaining: 39.3s
23:	learn: 0.4410965	total: 962ms	remaining: 39.1s
24:	learn: 0.4390534	total: 1000ms	remaining: 39s
25:	learn: 0.4372121	total: 1.04s	remaining: 38.9s
26:	learn: 0.4354268	total: 1.08s	remaining: 38.8s
27:	learn: 0.4336744	total: 1.12s	remaining: 38.9s
28:	learn: 0.4320755	total: 1.17s	remaining: 39.1s
29:	learn: 0.4307104	total: 1.21s	remaining: 39.1s
30:	learn: 0.4294751	total: 1.25s	remaining: 38.9s
31:	learn: 0.4282051	total: 1.28s	remaining: 38.9s
32:	learn: 0.4271240	total: 1.33s	remaining: 38.9s
33:	learn: 0.4260481	total: 1.36s	

In [ ]:
#from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score
#error= []
#test_yhat= []
#fold= StratifiedKFold(n_splits= 2)
#i= 1
#for train_ind, test_ind in fold.split(X,y):
  #train_x, val_x = x.iloc[train_ind], x.iloc[train_ind]
  #train_y, val_y = y.iloc[test_ind], y.iloc[test_ind]
                           
  #model= CatBoostClassifier(n_estimators= 3000, eval_metric= 'Accuracy', learning_rate = 0.1, random_seed= 1234)
  #model.fit(train_x, train_y, eval_set=[(val_x, val_y)], early_stopping_rounds= 300, verbose=100)
  #preds= model.predict(val_x)           
  #print('err:', accuracy_score(val_y, preds))
  #error.append(accuracy_score(val_y, preds))
  #cvc= cross_val_score(model, X, y, cv= 5)
  #bin_clf_rep = classification_report(val_y,preds, zero_division=1)
  #print(cvc.mean())
  #print(bin_clf_rep)
   
   
  #yhat= model.predict(test)
  #test_yhat.append(yhat)
#np.mean(error)   

In [37]:
lgb= LGBMClassifier()
lgb.fit(train_x,train_y)
preds = lgb.predict(val_x)
prob= lgb.predict_proba(val_x)

roc_score = roc_auc_score(val_y, prob[:,1])
cv_score = cross_val_score(lgb,X,y,cv=5).mean()
score = accuracy_score(val_y,preds)
bin_clf_rep = classification_report(val_y,preds, zero_division=1)
print("Accuracy :", score)
print("CV Score :", cv_score)
print("AUC Score : ", roc_score)
print(bin_clf_rep)
#print(confusion_matrix(y_test,y_pred))

lgb

Accuracy : 0.8089285714285714
CV Score : 0.7727857142857143
AUC Score :  0.837149467743027
              precision    recall  f1-score   support

           0       0.84      0.92      0.88     12686
           1       0.66      0.46      0.54      4114

    accuracy                           0.81     16800
   macro avg       0.75      0.69      0.71     16800
weighted avg       0.80      0.81      0.80     16800



NameError: ignored

In [ ]:
lgbm_params = {"n_estimators" : [100, 200, 500, 1000, 2000],
               "subsample" : [0.6, 0.8, 1.0],
               "learning_rate" : [0.1, 0.01, 0.02, 0.03],
               "min_child_samples" : [5, 10, 20, 30]}

In [ ]:
from sklearn.model_selection import GridSearchCV
lgbm_cv_model = GridSearchCV(lgb, 
                             lgbm_params, 
                             cv = 5,
                             verbose = 1,
                             n_jobs = -1)

In [38]:
##lgbm_cv_model.fit(train_x, train_y)

In [39]:
#lgbm_cv_model.best_params_

In [40]:
lgbm_m = LGBMClassifier(learning_rate = 0.01,
                            min_child_samples = 30,
                            n_estimators = 1000,
                            subsample = 0.6)
lgbm_m.fit(train_x,train_y)
y_pred = lgbm_m.predict(val_x)
proba = lgbm_m.predict_proba(val_x)
roc_score = roc_auc_score(val_y, proba[:,1])
cv_score = cross_val_score(lgbm_m,X, y ,cv=5).mean()
score = accuracy_score(val_y,y_pred)
bin_clf_rep = classification_report(val_y,y_pred, zero_division=1)
#print(name[0:name.find("(")])
print("Accuracy :", score)
print("CV Score :", cv_score)
print("AUC Score : ", roc_score)
print(bin_clf_rep)
#print(confusion_matrix(y_test,y_pred))

Accuracy : 0.8106547619047619
CV Score : 0.7681250000000001
AUC Score :  0.8398608673765675
              precision    recall  f1-score   support

           0       0.84      0.93      0.88     12686
           1       0.66      0.46      0.54      4114

    accuracy                           0.81     16800
   macro avg       0.75      0.69      0.71     16800
weighted avg       0.80      0.81      0.80     16800



In [41]:
lgbm_m.fit(X,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=30, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.6, subsample_for_bin=200000, subsample_freq=0)

In [42]:
test_yhat= lgbm_m.predict_proba(test)[:, 1]

In [43]:
sub= pd.read_csv('/content/drive/My Drive/Bootcamp2020/SampleSubmission.csv')
sub.head()

,Applicant_ID,default_status
0,Apcnt_1000032,1
1,Apcnt_1000048,1
2,Apcnt_1000052,1
3,Apcnt_1000076,1
4,Apcnt_1000080,1


In [44]:
sub.shape

(24000, 2)

In [45]:
sub['default_status'] = test_yhat  
sub

,Applicant_ID,default_status
0,Apcnt_1000032,0.283337
1,Apcnt_1000048,0.311346
2,Apcnt_1000052,0.386617
3,Apcnt_1000076,0.715723
4,Apcnt_1000080,0.136542
...,...,...
23995,Apcnt_999940,0.725564
23996,Apcnt_999956,0.301470
23997,Apcnt_999976,0.261286
23998,Apcnt_999984,0.530947


In [46]:
sub.to_csv('run.csv', index= False)